<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
kr_dst run notebook</p><br>

# Runs the kr_dst

In [49]:
import os
import sys
import glob
import logging
import math
import numpy  as np
import warnings
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from   dataclasses import dataclass

In [50]:
from nextflex.core import Setup
from nextflex.core import PosQ

In [51]:
from nextflex.core import get_evt_true_positions_and_energy
from nextflex.core import get_evt_true_positions_df
from nextflex.core import get_sensor_response
from nextflex.core import sensor_response_ti
from nextflex.core import event_sensor_response_ti
from nextflex.core import sensor_number_response_ti

In [52]:
from nextflex.core import mcparts_and_sensors_response
from nextflex.core import get_s1
from nextflex.core import get_s2
from nextflex.core import get_qtot

In [53]:
from nextflex.core import get_q
from nextflex.core import get_pos
from nextflex.core import get_position
from nextflex.core import diff_pos

In [80]:
from nextflex.krypton_dst import kr_dst
from nextflex.krypton_dst import kr_join_dst
from nextflex.krypton_dst import prepare_tmpdir

In [55]:
from nextflex.krypton_analysis import histo_df
from nextflex.krypton_analysis import histos_df
from nextflex.krypton_analysis import histo1d
from nextflex.krypton_analysis import histos1d

In [56]:
import invisible_cities.core.system_of_units  as units
from invisible_cities.core.core_functions     import in_range

from invisible_cities.io.mcinfo_io import get_sensor_binning
from invisible_cities.io.mcinfo_io import load_mcparticles_df
from invisible_cities.io.mcinfo_io import load_mcsensor_response_df

In [57]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Run configuration

In [58]:
FDATA = os.environ['FLEXDATA']
print(f'path to data ={FDATA}')

path to data =/Users/jj/Development/flexdata


### Setup

***
@dataclass\
class Setup:
    
    flexDATA          : str   = "/Users/jj/Development/flexdata"
    sipmPDE           : float = 1.0
    maskPDE           : float = 1.0
    qTh               : float = 0.0
    tpConfig          : str   = "FLEX100_M6_O6" # means thickness 6 mm hole 6mm
    mapDIR            : str   = "flexmaps" # where to find the SiPM map
    fibres            : bool  = False

***


In [73]:
setup = Setup(flexDATA = FDATA,
              sipmPDE  = 1.0,
              maskPDE  = 1.0,
              qTh      = 0.0,
              tpConfig = "FLEX100_M6_O6")
print(setup)


        Setup <FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0>:
        tracking plane configuration = FLEX100_M6_O6
        sipm PDE                     = 1.0
        transmission of teflon masks = 1.0
        charge threshold             = 0.0
        energy sensors               = PMTs
        pitch                        = 15.55
        number of energy sensors     = 60
        number of SiPMs              = 3093
        root directory               = /Users/jj/Development/flexdata/FLEX100_M6_O6
        number of h5 files in dir    = 100
        sipm map at                  = /Users/jj/Development/flexdata/flexmaps/sipm_map_15.6_mm.csv
        sipm map name                = sipm_map_15.6_mm.csv
        output file                  = /Users/jj/Development/flexdata/kdsts/FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0.csv
        


### SiPM map

In [64]:
setup.mPath

'/Users/jj/Development/flexdata/flexmaps/sipm_map_15.6_mm.csv'

In [65]:
sipm_map      = pd.read_csv(setup.mPath)

In [66]:
sipm_map.head()

,Unnamed: 0,sensor_id,sensor_name,x,y,xl,xr,yu,yd,id_xl,id_xr,id_yu,id_yd
0,0,1190,TP_SiPM,-202.15,-373.20,-217.699994,-186.599994,-357.650012,-388.750012,1189,1191,1232,1151
1,1,1195,TP_SiPM,-124.40,-373.20,-139.950002,-108.850002,-357.650012,-388.750012,1194,1196,1237,1156
2,2,1073,TP_SiPM,124.40,-435.40,108.850002,139.950002,-419.849994,-450.949994,1072,1074,1103,1047
3,3,1159,TP_SiPM,-77.75,-388.75,-93.300000,-62.200000,-373.200000,-404.300000,1158,1160,1198,1123
4,4,2068,TP_SiPM,295.45,-124.40,279.900012,311.000012,-108.850002,-139.950002,2067,2069,2129,2007


### Directory for files describing the SiPM plane configuration

In [14]:
ifnames = glob.glob(f"{FDATA}/{setup.tpConfig}/*.h5")
print(f'found {len(ifnames)} files')

found 100 files


### Extract kr_dsts from h5

In [67]:
gf, bf = kr_dst(sipm_map, setup, ic=20)
print(f'good files ={len(gf)}')
print(f'bad files ={len(bf)}')


reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1023.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M6_O6.Kr83.ACTIVE.1023.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1032.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M6_O6.Kr83.ACTIVE.1032.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1031.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M6_O6.Kr83.ACTIVE.1031.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1024.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M

### Join kr_dsts 

In [68]:
ifnames2 = glob.glob(f"{setup.tmpdir}/*.csv")

In [69]:
print(f'found {len(ifnames2)} files')

found 100 files


In [70]:
ofile_name = f"{setup.name}.csv"
print(f"output file name = {ofile_name}")

output file name = FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0.csv


In [74]:
ofile = f"{FDATA}/kdsts/{ofile_name}"
print(f"Path to ofile = {setup.ofile}")

Path to ofile = /Users/jj/Development/flexdata/kdsts/FLEX100_M6_O6_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0.csv


In [76]:
krdf, BF = kr_join_dst(ifnames2, verbose=False)
print(f'bad files ={len(BF)}')
print(krdf)
krdf.to_csv(ofile)

bad files =0
    event_id      true_x      true_y      true_z         KE  S1e   S2e  \
0     104500 -290.299380  283.587830  1175.99710  53.914030   29  7991   
1     104501  419.138370   -0.582073   829.08356  41.543297   21  8009   
2     104502  157.880660  247.045960   425.46478  41.543297   17  8333   
3     104503 -222.691010 -203.131680   920.13570  41.543297   19  8602   
4     104504 -206.557970 -150.945130   111.76873  53.873820   15  8476   
..       ...         ...         ...         ...        ...  ...   ...   
95    102095 -444.513280  135.277280    36.82412  41.543297   12  7511   
96    102096 -161.594090 -178.331570   726.26465  53.953620   21  8904   
97    102097  301.075870 -208.838970   488.60150  50.821550   17  8093   
98    102098  -16.136173  428.891540   433.23752  41.543297   16  7825   
99    102099 -215.592990 -389.331180   351.16280  43.028070   12  7694   

      xMax    yMax        xPos  ...   Qtot  qMax    qL    qR    qU    qD  \
0  -295.45  279.90 -29

In [77]:
ofile == setup.ofile

True

## FLEX100_M6_O6_P10.EL8bar

In [78]:
setup = Setup(flexDATA   = FDATA,
                  sipmPDE    = 1.0,
                  maskPDE    = 1.0,
                  qTh        = 0.0,
                  tpConfig   = "FLEX100_M6_O6_P10.EL8bar")
print(setup)


        Setup <FLEX100_M6_O6_P10.EL8bar_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0>:
        tracking plane configuration = FLEX100_M6_O6_P10.EL8bar
        sipm PDE                     = 1.0
        transmission of teflon masks = 1.0
        charge threshold             = 0.0
        energy sensors               = PMTs
        pitch                        = 10.0
        number of energy sensors     = 60
        number of SiPMs              = 7484
        root directory               = /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar
        number of h5 files in dir    = 100
        sipm map at                  = /Users/jj/Development/flexdata/flexmaps/sipm_map_10.0_mm.csv
        sipm map name                = sipm_map_10.0_mm.csv
        output file                  = /Users/jj/Development/flexdata/kdsts/FLEX100_M6_O6_P10.EL8bar_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0.csv
        


In [82]:
prepare_tmpdir(setup)

creating dir /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.EL8bar_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0


In [83]:
sipm_map      = pd.read_csv(setup.mPath)

In [84]:
sipm_map.head()

,Unnamed: 0,sensor_id,sensor_name,x,y,xl,xr,yu,yd,id_xl,id_xr,id_yu,id_yd
0,0,4715,TP_SiPM,225.0,-5.0,215.0,235.0,5.0,-15.0,4714,4716,4813,4617
1,1,4430,TP_SiPM,315.0,-35.0,305.0,325.0,-25.0,-45.0,4429,4431,4528,4332
2,2,2828,TP_SiPM,285.0,-205.0,275.0,295.0,-195.0,-215.0,2827,2829,2917,2740
3,3,4332,TP_SiPM,315.0,-45.0,305.0,325.0,-35.0,-55.0,4331,4333,4430,4235
4,4,4235,TP_SiPM,315.0,-55.0,305.0,325.0,-45.0,-65.0,4234,4236,4332,4139


In [85]:
gf, bf = kr_dst(sipm_map, setup, ic=20)

reading file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.Kr83.ACTIVE.3099.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.EL8bar_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M6_O6_P10.Kr83.ACTIVE.3099.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.Kr83.ACTIVE.3091.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.EL8bar_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M6_O6_P10.Kr83.ACTIVE.3091.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.Kr83.ACTIVE.3050.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.EL8bar_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0/FLEX100_M6_O6_P10.Kr83.ACTIVE.3050.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6_P10.EL8bar/FLEX100_M6_O6_P10.Kr83

In [86]:
print(f'good files ={len(gf)}')
print(f'bad files ={len(bf)}')

good files =100
bad files =0


In [87]:
ifnames2 = glob.glob(f"{setup.tmpdir}/*.csv")
print(f'found {len(ifnames2)} files')

found 100 files


In [88]:
krdf, BF = kr_join_dst(ifnames2, verbose=False)
print(f'bad files ={len(BF)}')

bad files =0


In [89]:
krdf.to_csv(setup.ofile)